 # **Preparing directories / Splitting data**

In [ ]:
import zipfile


zip_files = ['test1', 'train']
# Will unzip the files so that you can see them..
for zip_file in zip_files:
    with zipfile.ZipFile("../input/dogs-vs-cats/{}.zip".format(zip_file),"r") as z:
        z.extractall(".")
        print("{} unzipped".format(zip_file))

In [ ]:
import os
import pandas as pd

filenames = os.listdir("/kaggle/working/train")
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append(1)
    else:
        categories.append(0)

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(df['filename'], df['category'], test_size=0.20)


In [ ]:
import shutil

current_dir = os.path.abspath(os.getcwd())
training_dir = os.path.join(current_dir, "training")
os.mkdir(training_dir)
cats_dir = os.path.join(training_dir, "cats")
dogs_dir = os.path.join(training_dir, "dogs")
os.mkdir(cats_dir)
os.mkdir(dogs_dir)

for i in range(len(X_train)):
    filename = X_train.iloc[i]
    folder = cats_dir if y_train.iloc[i] == 0 else dogs_dir
    dst = os.path.join(folder,filename)
    os.rename("/kaggle/working/train/{}".format(filename), dst)

In [ ]:
validation_dir = training_dir = os.path.join(current_dir, "validation")
os.mkdir(validation_dir)
cats_dir_v = os.path.join(validation_dir, "cats")
dogs_dir_v = os.path.join(validation_dir, "dogs")
os.mkdir(cats_dir_v)
os.mkdir(dogs_dir_v)

for i in range(len(X_valid)):
    filename = X_valid.iloc[i]
    folder = cats_dir_v if y_valid.iloc[i] == 0 else dogs_dir_v
    dst = os.path.join(folder,filename)
    os.rename("/kaggle/working/train/{}".format(filename), dst)

In [ ]:
shutil.rmtree("train")

# **Model**

In [ ]:
from keras import layers 
from keras import models
from keras import optimizers 

model = models.Sequential() 
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3))) 
model.add(layers.MaxPooling2D((2, 2))) 
model.add(layers.Conv2D(64, (3, 3), activation='relu')) 
model.add(layers.MaxPooling2D((2, 2))) 
model.add(layers.Conv2D(128, (3, 3), activation='relu')) 
model.add(layers.MaxPooling2D((2, 2))) 
model.add(layers.Conv2D(128, (3, 3), activation='relu')) 
model.add(layers.MaxPooling2D((2, 2))) 
model.add(layers.Flatten()) 
model.add(layers.Dropout(0.5)) 
model.add(layers.Dense(512, activation='relu')) 
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])



In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

**Data pre-processing steps**
1.  Read the picture files. 
2. Decode the JPEG content to RGB grids of pixels. 
3. Convert these into floating-point tensors. 
4. Rescale the pixel values (between 0 and 255) to the [0, 1] interval (neural networks prefer to deal with small input values)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255) 
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory( training_dir, target_size=(150, 150),batch_size=20, class_mode='binary')
validation_generator = test_datagen.flow_from_directory( validation_dir,target_size=(150, 150), batch_size=20, class_mode='binary')


***ImageGenerator***

fit_generator method, the equivalent of fit for data generators like this one. It expects as its first argument a Python generator that will yield batches of inputs and targets indefinitely. Because the data is being generated endlessly, the Keras model needs to know how many samples to draw from the generator before declaring an epoch over. 

This is the role of the steps_per_epoch argument: after having drawn **steps_per_epoch** batches from the generator—that is, after having run for **steps_per_epoch** gradient descent steps—the fitting process will go to the next epoch. 
In this case, batches are 20 samples, so it will take 100 batches until you see your target of 2,000 samples.

When using **fit_generator**, you can pass a **validation_data** argument, much as with the fit method. It’s important to note that this argument is allowed to be a *data generator*, but it could also be a *tuple of Numpy arrays*. 
 
 !!! If you pass a generator as validation_data, then this generator is expected to yield batches of validation data *endlessly*; thus you should also specify the **validation_steps** argument, which tells the process how many batches to draw from the validation generator for evaluation.

In [ ]:
train_datagen = ImageDataGenerator( rescale=1./255, rotation_range=40, width_shift_range=0.2, 
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True,)
test_datagen = ImageDataGenerator(rescale=1./255) #validation data shouldn't be augmented!!!
train_generator = train_datagen.flow_from_directory( training_dir, target_size=(150, 150), batch_size=32, class_mode='binary') 
#Because you use binary_crossentropy loss, you need binary labels.
validation_generator = test_datagen.flow_from_directory( validation_dir, target_size=(150, 150), batch_size=32, class_mode='binary')
history = model.fit_generator( train_generator, steps_per_epoch=100, epochs=100, validation_data=validation_generator, validation_steps=50)


In [ ]:
model.save_weights("model.h5")

In [ ]:
test_dir = os.path.join(current_dir, 'test1')
test_gen = ImageDataGenerator(rescale=1./255)
test_filenames = os.listdir(test_dir)
dst_dir = os.path.join(test_dir,'sub')
os.mkdir(dst_dir)
#flowfromdirectory treats the sub-folder as the label of the images it contains. So it only search images in sub-folders.
for filename in test_filenames:
    dst = os.path.join(dst_dir,filename)
    src = os.path.join(test_dir,filename)
    os.rename(src, dst)


test_generator = test_gen.flow_from_directory(directory=test_dir, target_size=(150, 150), batch_size=32)

In [ ]:
predict = model.predict_generator(test_generator, 100)

In [ ]:
submission_df = test_df.copy()
submission_df['id'] = submission_df['filename'].str.split('.').str[0]
submission_df['label'] = pd.DataFrame(predict)
submission_df.drop(['filename'], axis=1, inplace=True)
submission_df.to_csv('submission.csv', index=False)